#### This notebook computes Reject Option Classification (ROC) post-processing algorithm baseline for bias mitigation.

In [40]:
%load_ext autoreload
%autoreload 2
%aimport logistic_regression_model
%aimport load_dataset
%aimport fairness_metrics

import sys
import os
import numpy as np
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import reject_option_classification
from IPython.display import Markdown, display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# INPUT PARAMS
LABEL_COL, PROTECT_COLS, MODE, START_EPOCH, NUM_EPOCH, ID, NUM_TRIALS, NUM_PROXIES, FILE_PATH, VERBOSE, \
LR_RATE, UPDATE, WEIGHTS_INIT, UPDATE_LR, BATCH_SIZE, BALANCE = "income", ["gender"], 0, 0, 40, -1, 1, 0, \
                                                                "../Datasets/adult_dataset/processed_adult.csv", 1, \
                                                                0.001, "cluster", 0, 10, 1000, 1

#### Computes and split the dataset into train, validation and test set

In [44]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

balanced = {"train_label_only": True, "test_label_only": False, "downsample": True} if BALANCE else None


df = load_dataset.get_data(FILE_PATH)
df = load_dataset.minmax_scale(df)

train_df, test_df = load_dataset.split_train_test(df, train=0.75)
val_df, test_df = load_dataset.split_train_test(test_df, train=0.5)

if balanced is not None:
    train_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["train_label_only"],
                            downsample=balanced["downsample"])
    val_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["train_label_only"],
                            downsample=balanced["downsample"])
    test_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["test_label_only"],
                            downsample=balanced["downsample"])

# Splitting dataset into train, test features
print("Statistics")
load_dataset.statistics(train_df, LABEL_COL, PROTECT_COLS, verbose=1)
load_dataset.statistics(val_df, LABEL_COL, PROTECT_COLS, verbose=1)
load_dataset.statistics(test_df, LABEL_COL, PROTECT_COLS, verbose=1)

train_dataset, train_dataset_pred = load_dataset.Dataset(train_df, LABEL_COL, PROTECT_COLS), load_dataset.Dataset(train_df, LABEL_COL, PROTECT_COLS)
val_dataset, val_dataset_pred = load_dataset.Dataset(val_df, LABEL_COL, PROTECT_COLS), load_dataset.Dataset(val_df, LABEL_COL, PROTECT_COLS)
test_dataset, test_dataset_pred = load_dataset.Dataset(test_df, LABEL_COL, PROTECT_COLS), load_dataset.Dataset(test_df, LABEL_COL, PROTECT_COLS)


# Metric used (should be one of allowed_metrics)
metric_name = "Average odds difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

print("---------- MAPPING ----------")
print("Train: ", train_dataset.mapping)
print("Val: ", val_dataset.mapping)
print("Test: ", test_dataset.mapping)
print("-----------------------------")


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Statistics
{0.0: gender
0.0    (4213, 0.376)
1.0    (6995, 0.624)
dtype: object, 1.0: gender
0.0    (1669, 0.149)
1.0    (9539, 0.851)
dtype: object}
label: 0.0: 11208 samples (50.00%)
label: 1.0: 11208 samples (50.00%)
{0.0: gender
0.0    (4213, 0.376)
1.0    (6995, 0.624)
dtype: object, 1.0: gender
0.0    (1669, 0.149)
1.0    (9539, 0.851)
dtype: object}
label: 0.0: 11208 samples (50.00%)
label: 1.0: 11208 samples (50.00%)
{0.0: gender
0.0    (1678, 0.503)
1.0    (1660, 0.497)
dtype: object, 1.0: gender
0.0    (1669, 0.5)
1.0    (1669, 0.5)
dtype: object}
label: 0.0: 3338 samples (50.00%)
label: 1.0: 3338 samples (50.00%)
---------- MAPPING ----------
Train:  {(1.0,): 0, (0.0,): 1}
Val:  {(1.0,): 0, (0.0,): 1}
Test:  {(0.0,): 0, (1.0,): 1}
-----------------------------


#### Trians sklearn logistic regression on training set

In [45]:
X_train = train_dataset.features
y_train = train_dataset.label

lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)

train_dataset_pred.label = y_train_pred

bcm = confusion_matrix(y_train_pred, y_train)
print(f"Accuracy {np.sum(y_train_pred == y_train)/len(y_train)*100}%, Balanced Acc: {0.5 * (fairness_metrics.true_positive_rate(bcm) + fairness_metrics.true_negative_rate(bcm)*100)}%")

Accuracy 82.36081370449678%, Balanced Acc: 42.63128999168545%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [46]:
favorable_label, unfavorable_label = 1, 0
pos_ind = np.where(lmod.classes_ == favorable_label)[0][0]

#### Computes model probabilities for validation and test set

In [47]:
X_valid = val_dataset_pred.features
y_valid = val_dataset_pred.label
val_scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

X_test = test_dataset_pred.features
y_test = test_dataset_pred.label
test_scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

### Find the optimal parameters from the validation set

#### Best threshold for classification only (no fairness)

In [48]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = (val_scores > class_thresh).reshape(-1)
    new_labels = val_dataset_pred.label.copy()
    new_labels[fav_inds] = favorable_label
    new_labels[~fav_inds] = unfavorable_label
    val_dataset_pred.label = new_labels.copy()
    
    
    # Optimizing for balanced acc
    bcm = confusion_matrix(val_dataset.label, val_dataset_pred.label)
    ba_arr[idx] = 0.5 * (fairness_metrics.true_positive_rate(bcm) + fairness_metrics.true_negative_rate(bcm))
    
    # Optimizing for acc
    # ba_arr[idx] = np.sum(val_dataset_pred.label == val_dataset.label)/len(val_dataset.label)

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.8238
Optimal classification threshold (no fairness constraints) = 0.5049


In [49]:
privileged_groups = [{'sex': 0}]
unprivileged_groups = [{'sex': 1}]

#### Estimate optimal parameters for the ROC method

In [50]:
ROC = reject_option_classification.RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(val_dataset, val_dataset_pred, val_scores)

In [51]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.5247
Optimal ROC margin = 0.1067


### Predictions from Validation Set

In [52]:
# Metrics for the test set
fav_inds = (val_scores > best_class_thresh).reshape(-1)
new_labels = val_dataset_pred.label.copy()
new_labels[fav_inds] = favorable_label
new_labels[~fav_inds] = unfavorable_label
val_dataset_pred.label = new_labels.copy()

display(Markdown("#### Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

acc = np.sum(val_dataset_pred.label.reshape(-1) == val_dataset.label.reshape(-1))/len(val_dataset.label)
odds = fairness_metrics.equalizing_odds(val_dataset_pred.label, val_dataset.label,
                                                       val_dataset.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(val_dataset.label == unfavorable_label), np.sum(val_dataset.label == favorable_label)]))

#### Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Accuracy: 82.38311920057102%
Equalizing Odds: [[0.712, 0.936], [0.876, 0.713]]
Weighted average odds difference 0.19350000000000003


In [53]:
# Transform the validation set
val_dataset_pred = ROC.predict(val_dataset_pred, val_scores)

display(Markdown("#### Validation set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))

acc = np.sum(val_dataset_pred.label.reshape(-1) == val_dataset.label.reshape(-1))/len(val_dataset.label)
odds = fairness_metrics.equalizing_odds(val_dataset_pred.label, val_dataset.label,
                                                       val_dataset.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(val_dataset.label == unfavorable_label), np.sum(val_dataset.label == favorable_label)]))

#### Validation set

##### Transformed predictions - With fairness constraints

Accuracy: 81.07601713062098%
Equalizing Odds: [[0.817, 0.913], [0.768, 0.769]]
Weighted average odds difference 0.04850000000000005


### Predictions from Test Set

In [54]:
test_dataset.protect =  np.logical_not(test_dataset.protect).astype(int)
test_dataset_pred.protect = np.logical_not(test_dataset_pred.protect).astype(int)

In [55]:
# Metrics for the test set
fav_inds = (test_scores > best_class_thresh).reshape(-1)
new_labels = test_dataset_pred.label.copy()
new_labels[fav_inds] = favorable_label
new_labels[~fav_inds] = unfavorable_label
test_dataset_pred.label = new_labels.copy()

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

acc = np.sum(test_dataset_pred.label.reshape(-1) == test_dataset.label.reshape(-1))/len(test_dataset.label)
odds = fairness_metrics.equalizing_odds(test_dataset_pred.label, test_dataset.label,
                                                       test_dataset.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(test_dataset.label == unfavorable_label), np.sum(test_dataset.label == favorable_label)]))

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Accuracy: 81.17136009586578%
Equalizing Odds: [[0.719, 0.939], [0.874, 0.713]]
Weighted average odds difference 0.1905


In [56]:
# Transform the validation set
test_dataset_pred = ROC.predict(test_dataset_pred, test_scores)

display(Markdown("#### Validation set"))+-
display(Markdown("##### Transformed predictions - With fairness constraints"))

acc = np.sum(test_dataset_pred.label.reshape(-1) == test_dataset.label.reshape(-1))/len(test_dataset.label)
odds = fairness_metrics.equalizing_odds(test_dataset_pred.label, test_dataset.label,
                                                       test_dataset.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(test_dataset.protect == unfavorable_label), np.sum(test_dataset.protect == favorable_label)]))

#### Validation set

##### Transformed predictions - With fairness constraints

Accuracy: 82.17495506291192%
Equalizing Odds: [[0.829, 0.924], [0.765, 0.769]]
Weighted average odds difference 0.04937732174955067
